In [25]:
# %pip install chartgpt
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
import pandas as pd
from chartgpt import ChartGPT

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/2014_usa_states.csv")
df.head()

,Rank,State,Postal,Population
0,1,Alabama,AL,4849377.0
1,2,Alaska,AK,736732.0
2,3,Arizona,AZ,6731484.0
3,4,Arkansas,AR,2966369.0
4,5,California,CA,38802500.0


In [27]:
cg = ChartGPT()
cg.load(df)
cg.plot("pop vs state")

In [28]:
from jupyter_dash import JupyterDash
import dash_mantine_components as dmc
from dash import dcc, html, Input, Output, State


def create_table(df):
    columns, values = df.columns, df.values
    header = [html.Tr([html.Th(col) for col in columns])]
    rows = [html.Tr([html.Td(cell) for cell in row]) for row in values]
    table = [html.Thead(header), html.Tbody(rows)]
    return table

table = create_table(df.head(10))


# Define the Dash application
app = JupyterDash(__name__)
app.layout = dmc.Stack(
    [
        dmc.TextInput(
            id="prompt",
            placeholder="Enter your prompt here",
            radius="xl",
            w=500,
            m="auto",
            rightSection=dmc.Button("Update", id="button", variant="gradient", radius="xl"),
        ),
        dmc.LoadingOverlay(
            dmc.Table(table),
            id="chart-container",
        ),
    ]
)

# Load dataframe into chartGPT
cg = ChartGPT()
cg.load(df)

# Callback on text updated
@app.callback(
    Output("chart-container", "children"),
    Input("button", "n_clicks"),
    State("prompt", "value"),
    prevent_initial_call=True,
)
def update_chart(n_clicks, prompt):
    fig = cg.plot(prompt, return_fig=True)
    children = dmc.Stack(
            [
                dcc.Graph(figure=fig),
                dmc.Prism(
                    cg.last_run_code,
                    language="python",
                    withLineNumbers=True,
                )
            ]
        )
    return children


if __name__ == "__main__":
    app.run_server(mode="inline")


Dash is running on http://127.0.0.1:8050/

